# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Hardware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
versioninfo()

In [ ]:
using Pkg

In [ ]:
Pkg.add("Revise")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")
Pkg.add(url="https://github.com/bwbioinfo/KEGGAPI.jl")

In [ ]:
using Revise
using DataFrames
using BenchmarkTools
using KEGGAPI

# Case 2: EC reaction information in Kegg Database



### 1. Identify reactions in KEGG database assiciated to a European Community number (EC.)

To determine if a reaction is part of KEGG database and obtain its information the function link takes as input the element the user is looking for, in this case "reaction", and the EC number of a reaction as "ec:X.XX.XX.XX"

In [ ]:
@time kegg_reaction = KEGGAPI.link("reaction", "ec:3.2.1.14")
DataFrame(
  kegg_reaction.data,
  kegg_reaction.colnames
)

  0.156340 seconds (231 allocations: 13.680 KiB)


Row,Target ID,Source ID
,String,String
1,ec:3.2.1.14,rn:R01206
2,ec:3.2.1.14,rn:R02334
3,ec:3.2.1.14,rn:R06081
4,ec:3.2.1.14,rn:R06082


### 2. Get reaction information

To obtain the reactions information, the "kegg_get" function requires an array of reaction KEGG identifier as "rn:RXXXX



In [ ]:
@time kegg_reaction_info = KEGGAPI.kegg_get(kegg_reaction.data[2])
split(kegg_reaction_info[2][1], "\n")

### 3. Identify the compounds related to the reaction

The input of the link function are two strings. The element the user is looking for, in this case a compound, and the reaction number as rn:RXXXXX.

In [ ]:
@time kegg_compound = KEGGAPI.link("compound", "rn:R01206")
DataFrame(
  kegg_compound.data,
  kegg_compound.colnames
)

### 4. Get compound information

To obtain the compund information, the "kegg_get" function requires an array of compound KEGG identifier as "cpd:CXXXXX

In [ ]:
@time kegg_compound_info = KEGGAPI.kegg_get(kegg_compound.data[2]);
split(kegg_compound_info[2][1], "\n")

### 5. Download and save a reaction image.

The get_image function is to download a any image, the imput is the reaction number as rn:RXXXXX

The save_image function is to save the figure in a png file. The input is a string wiht the name of the file and the extension ".png"

In [ ]:
@time kegg_image = KEGGAPI.get_image("rn:R01206")
@time KEGGAPI.save_image(kegg_image, "r01206.png")

### 6. Visualization of the reaction image

To visualize the saved image, load the png file with fileIO package option.

In [ ]:
Pkg.add("TestImages")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("Colors")

In [ ]:
using Images, TestImages, Colors
img = load("r01206.png")

### 7. Get Kegg Ortholog group

To get the ortholog group use the link function with the string "ko" and the reaction number or concatenate string as "RXXXXX"

In [ ]:
kegg_orthogroup = KEGGAPI.link("ko", "R01206")
DataFrame(
  kegg_orthogroup.data,
  kegg_orthogroup.colnames
)